# CS 6290: Privacy-enhancing Technologies
## Assignment 1

(Acknowledgement: this assignment is adapted from [Homework 9 for UVM CS 3990, developed by Joseph P. Near](https://github.com/jnear/cs3990-secure-computation/blob/main/homework/CS3990_HW_9.ipynb).)

In [218]:
import numpy as np
import random
import hashlib

## The Petersen Graph

[The Petersen Graph](https://en.wikipedia.org/wiki/Petersen_graph) has 10 vertices and 15 edges, and can be colored with 3 colors.

![alt text](https://upload.wikimedia.org/wikipedia/commons/thumb/9/90/Petersen_graph_3-coloring.svg/220px-Petersen_graph_3-coloring.svg.png "Title")

In [219]:
coloring = {
    # outer five nodes, clockwise from top
    0: 'red',
    1: 'blue', 
    2: 'green',
    3: 'red',
    4: 'blue',
    # inner five nodes, clockwise from top
    5: 'blue', 
    6: 'red',
    7: 'red',
    8: 'green',
    9: 'green'
}
print('Number of nodes:', len(coloring))

Number of nodes: 10


In [220]:
edges = [
    # outer shape, clockwise from top
    (0, 1),
    (1, 2),
    (2, 3),
    (3, 4),
    (4, 0),
    # inner shape, clockwise from top
    (5, 0), (5, 7),
    (6, 1), (6, 8),
    (7, 2), (7, 9),
    (8, 3), (8, 5),
    (9, 4), (9, 6)
]
print('Number of edges:', len(edges))

Number of edges: 15


## Background

**Your goal in this assignment** is to implement a zero-knowledge interactive proof protocol that allows the Prover to convince the Verifier that the Prover knows a valid 3-coloring for the Petersen graph - without revealing the coloring.

You can find the description in [Matt Green's blog post](https://blog.cryptographyengineering.com/2014/11/27/zero-knowledge-proofs-illustrated-primer/).

For those interested in a more theoretical understanding of Zero-Knowledge Proofs and the 3-coloring problem, we recommend exploring these learning materials:
- **Interactive Zero-Knowledge Proofs:** Stanford University, CS355 Course Notes, [https://crypto.stanford.edu/cs355/18sp/lec3.pdf](https://crypto.stanford.edu/cs355/18sp/lec3.pdf)
- **3-Coloring Zero-Knowledge Proof:** University of Illinois at Urbana-Champaign, CS498 Course Notes (Section 15.1), [https://courses.grainger.illinois.edu/cs498ac3/fa2020/Files/Lecture_15_Scribe.pdf](https://courses.grainger.illinois.edu/cs498ac3/fa2020/Files/Lecture_15_Scribe.pdf)

## Protocol

The Prover and Verifier perform the following steps $n^2$ times, where $n$ is the number of vertices in the graph:

- **Step 1: shuffle and commit.** The Prover randomizes the colors and commits to the coloring. The Prover sends the commitment to the Verifier.
- **Step 2: challenge.** The Verifier picks a random edge in the graph.
- **Step 3: response.** The Prover opens the commitment for the two vertices connected by the chosen edge. If the two colors are the same, the Verifier rejects.

## Question 1 (7 points)

Implement the above protocol for an interactive zero-knowledge proof of the graph coloring. (**6 points**)

In [221]:
class Prover:
    def __init__(self, coloring):
        self.original_coloring = coloring
        self.shuffled_colors = {}
        self.commitments = {}
    
    def shuffle_and_commit(self):
        # 生成颜色置换
        color_map = {'red':0, 'blue':1, 'green':2}
        permutation = list(np.random.permutation([0,1,2]))
        
        # 转换颜色并打乱,这里新的颜色只是为了掩盖原来的颜色
        shuffled = {}
        for v in self.original_coloring:
            original_color = color_map[self.original_coloring[v]]
            new_color_idx = permutation[original_color]
            shuffled[v] = ['red', 'blue', 'green'][new_color_idx]
        
        # 生成承诺
        commitments = {}
        for v in shuffled:
            data = f"{v}{shuffled[v]}"
            commitments[v] = hashlib.sha256(data.encode()).hexdigest()
        
        self.shuffled_colors = shuffled
        self.commitments = commitments
        return commitments
    
    def response(self, edge, V):
        v1, v2 = edge
        c1, c2 = self.shuffled_colors[v1], self.shuffled_colors[v2]
        return V.check(edge, c1, c2)

class Verifier:
    def __init__(self, graph_edges):
        self.edges = graph_edges
        self.commitments = {}
    
    def receive_commitment(self, commitments):
        self.commitments = commitments
    
    def challenge(self):
        return random.choice(self.edges)
    
    def check(self, edge, c1, c2):
        v1, v2 = edge
        # 验证颜色不同
        if c1 == c2:
            return False
        
        # 验证承诺正确性
        data1 = f"{v1}{c1}"
        data2 = f"{v2}{c2}"
        return (self.commitments[v1] == hashlib.sha256(data1.encode()).hexdigest() and
                self.commitments[v2] == hashlib.sha256(data2.encode()).hexdigest())

def run_protocol(coloring, iterations=len(edges)**2):
    P = Prover(coloring)
    V = Verifier(edges)
    for _ in range(iterations):
        commitments = P.shuffle_and_commit()
        V.receive_commitment(commitments)
        challenge_edge = V.challenge()
        check_result = P.response(challenge_edge, V)
        if not check_result:
            return False
    return True

# 测试有效着色
print("wheteher satifies 3-color:", run_protocol(coloring))

wheteher satifies 3-color: True


**Note:** You should modify `edges` to demonstrate the Verifier will reject the proof with an invalid coloring. (**1 point**).

In [222]:
# 测试无效着色（将顶点0和1设为相同颜色）
invalid_coloring = coloring.copy()
invalid_coloring[0] = 'blue'  # 原为red，现在与顶点1同色
print("wheteher satifies 3-color:", run_protocol(invalid_coloring))

wheteher satifies 3-color: False


## Question 2 (1 points)

In 2-5 sentences, argue that the protocol is zero-knowledge (it doesn't reveal anything about the witness).

YOUR ANSWER HERE
- the verifier cannot obtain any information about the real coloring through the protocol interaction. The verifier can only observe the hashed values of the commitments and the color pairs of randomly selected edges. Since the colors are randomly permuted, each interaction's commitment is independently random, and the verifier can only view the randomly selected edges, making it impossible to infer the colors of other vertices.
- Moreover, the verifier can generate valid interaction records through a simulator without using the real coloring, which proves that the protocol does not reveal any additional information.

## Question 3 (2 points)

What is the probability that a cheating Prover is *not* caught in this protocol, and why? (You should give the proof of your answer)

- If there is at least one illegal edge (with the same color), the probability that the verifier selects this edge in each challenge is $\frac{1}{15}$. 
- The probability of passing each verification(not be caught) is $\frac{14}{15}$. 
- After k independent verifications, the probability of passing all of them(not be caught) is $(\frac{14}{15})^k$.When $k = 15^2 = 225$, the probability is $(\frac{14}{15})^{225}\approx2.3\times10^{-7}$, which is close to zero.